In [34]:
import networkx as nx
import numpy as np
import random
import math
from collections import Counter
import io

# --- 1. LOAD NETWORKX GRAPH ---
print("Loading Zachary's Karate Club Graph...")
G = nx.karate_club_graph()

# --- 2. CREATE INDEX MAPPING ---
# Ensure the node IDs are sequential from 0 to N-1 for array processing
node_list = sorted(G.nodes())
node_to_index = {node: i for i, node in enumerate(node_list)}
index_to_node = {i: node for node, i in node_to_index.items()}
NUM_NODES = G.number_of_nodes() # n in your theory
DIMENSION = NUM_NODES

# --- 3. HYPERPARAMETERS (Adjusted for a small graph) ---
NUM_PARTICLES = 50     # P_n: Total population size
MAX_ITERATIONS = 50    # N_max: Maximum iterations

# PSO parameters (standard)
W_MAX = 0.9
W_MIN = 0.4
C1 = 2.0
C2 = 2.0
V_MAX = 6.0

# OBL/Crossover parameters
A_i = 0                # Lower bound for node assignment (0-based indexing for cluster IDs)
B_i = NUM_NODES - 1    # Upper bound for node assignment (Max possible cluster ID)
NUM_CLUSTERS = 4       # Target number of communities (K) - Common for this network
P_c = 0.2              # Ratio of crossover (20% of population)

print(f"Graph Loaded: Karate Club has {NUM_NODES} nodes.")

Loading Zachary's Karate Club Graph...
Graph Loaded: Karate Club has 34 nodes.


In [35]:
def calculate_modularity(G, community_assignment_array):
    """Calculates the Modularity (Q) of a community assignment."""

    node_assignment = {}
    for i in range(DIMENSION):
        # Map array index 'i' (0 to N-1) to original node ID
        node_id = index_to_node[i]

        # Get the community ID assigned by the particle
        community_id = community_assignment_array[i]

        # Assign the community ID to the original node ID
        node_assignment[node_id] = community_id

    # Convert to NetworkX partition format: list of sets of nodes
    communities = {}
    for node, c_id in node_assignment.items():
        if c_id not in communities:
            communities[c_id] = set()
        communities[c_id].add(node)

    partition = list(communities.values())

    if not partition or all(not c for c in partition):
        return 0.0

    try:
        Q = nx.community.modularity(G, partition)
    except nx.exception.NotAPartition:
        return -1.0
    except ZeroDivisionError:
        return 0.0

    return Q

In [36]:
# --- MAIN HYBRID PSO LOOP ---
for iter in range(MAX_ITERATIONS):
    # Dynamic Inertia Weight
    w = W_MAX - (W_MAX - W_MIN) * iter / MAX_ITERATIONS

    # ----------------------------------------------------
    # PHASE 1: STANDARD PSO UPDATES (Movement)
    # ----------------------------------------------------

    # 1. Update Velocity and Position (DPSO Steps)
    for particle in swarm:
        # Standard PSO Movement... (same as before)
        r1 = np.random.rand(DIMENSION)
        r2 = np.random.rand(DIMENSION)

        cognitive_term = C1 * r1 * (particle.pbest_position - particle.position)
        social_term = C2 * r2 * (gbest_position - particle.position)

        new_velocity = w * particle.velocity + cognitive_term + social_term
        new_velocity = np.clip(new_velocity, -V_MAX, V_MAX)
        particle.velocity = new_velocity

        # Update Position (Discretization)
        for i in range(DIMENSION):
            p_switch = 1.0 / (1.0 + math.exp(-particle.velocity[i]))

            if random.random() < p_switch:
                pbest_assignment = particle.pbest_position[i]
                gbest_assignment = gbest_position[i]

                if random.random() < 0.5:
                    particle.position[i] = gbest_assignment
                else:
                    particle.position[i] = pbest_assignment

            particle.position[i] = int(particle.position[i] % NUM_CLUSTERS)

        particle.current_fitness = calculate_modularity(G, particle.position)

    # 2. Update Pbest and Gbest after movement
    for particle in swarm:
        if particle.current_fitness > particle.pbest_fitness:
            particle.pbest_fitness = particle.current_fitness
            particle.pbest_position = particle.position.copy()

        if particle.pbest_fitness > gbest_fitness:
            gbest_fitness = particle.pbest_fitness
            gbest_position = particle.pbest_position.copy()

    # ----------------------------------------------------
    # PHASE 2: CUSTOM CROSSOVER OPERATION (Elitism)
    # ----------------------------------------------------

    # Sort particles by current fitness in descending order to identify top parents
    swarm.sort(key=lambda p: p.current_fitness, reverse=True)

    num_crossover = int(NUM_PARTICLES * P_c)
    if num_crossover % 2 != 0: num_crossover = max(2, num_crossover + 1) # Ensure even and at least 2

    crossover_parents = swarm[:num_crossover]
    new_offspring_swarm = []

    # Paired Crossover: I1 & I_last, I2 & I_(last-1), etc.
    for j in range(num_crossover // 2):
        parent_A = crossover_parents[j]
        parent_B = crossover_parents[num_crossover - 1 - j]

        split_point = random.randint(1, DIMENSION - 1)

        # Create two offspring positions
        offspring_1_pos = np.concatenate((parent_A.position[:split_point], parent_B.position[split_point:]))
        offspring_2_pos = np.concatenate((parent_B.position[:split_point], parent_A.position[split_point:]))

        # Create new particles and evaluate their fitness
        offspring_1 = Particle(DIMENSION, NUM_CLUSTERS, initial_position=offspring_1_pos)
        offspring_1.current_fitness = calculate_modularity(G, offspring_1_pos)

        offspring_2 = Particle(DIMENSION, NUM_CLUSTERS, initial_position=offspring_2_pos)
        offspring_2.current_fitness = calculate_modularity(G, offspring_2_pos)

        new_offspring_swarm.extend([offspring_1, offspring_2])

    # 3. Elitist Selection (Combine Parents + Offspring and select the best N)
    if new_offspring_swarm:
        combined_population = swarm + new_offspring_swarm
        combined_population.sort(key=lambda p: p.current_fitness, reverse=True)
        # The new swarm is the best N_PARTICLES from the combined pool
        swarm = combined_population[:NUM_PARTICLES]

    # Final Gbest check from the newly selected swarm
    current_gbest = max(swarm, key=lambda p: p.current_fitness)
    if current_gbest.current_fitness > gbest_fitness:
        gbest_fitness = current_gbest.current_fitness
        gbest_position = current_gbest.position.copy()

    print(f"Iteration {iter+1}/{MAX_ITERATIONS}: Global Best Modularity = {gbest_fitness:.4f}")

# --- FINAL RESULT ---
print("\n--- OPTIMIZATION COMPLETE ---")
print(f"Final Best Modularity Score: {gbest_fitness:.4f}")

final_assignments = {index_to_node[i]: gbest_position[i] for i in range(DIMENSION)}
community_counts = Counter(final_assignments.values())
print(f"Number of distinct communities found: {len(community_counts)}")
print("Top 5 largest communities (size):", community_counts.most_common(5))

ValueError: operands could not be broadcast together with shapes (34,) (62,) 